# Welche Dateitypen gibt es?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [1]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [6]:
import pandas as pd

query ="MATCH (f:Git:File) RETURN f.relativePath as relativePath"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [7]:
df.head()

,relativePath
0,None
1,src/test/java/junit/samples/ListTest.java
2,src/main/java/org/junit/rules/ExpectedExceptio...
3,src/test/java/org/junit/rules/ExpectedExceptio...
4,src/test/java/org/junit/tests/running/methods/...


Der folgende Codeabschnitt extrahiert die verschiedenen Dateitypen entsprechend der Dateiendung und zählt deren Häufigkeit. Die Dateitypen werden in der Variable <font face="Courier">datatype</font> und deren Häufigkeit in der Variable <font face="Courier">frequency</font> gespeichert.

In [8]:
# zu implementieren


## 4. Visualisierung
Die Daten werden mittels eines Pie Charts visualisiert (https://plot.ly/python/pie-charts/, labels=datatype, values=frequency).

In [ ]:
# zu implementieren

